# Analysis Notebook for Template Technology

In [ ]:
import os
import sys

import numpy             as np
import matplotlib.pyplot as pl
import pandas            as pd
import seaborn           as sb

sys.path.insert(0, os.path.abspath("../../../src"))

import tyche             as ty

## Load data and compile technology models

Every decision context analysis in Tyche will begin with three commands: instantiate the `Designs` class, instantiate the `Investments` class, and compile the technology model functions and data in the `Designs` class. These commands read in and validate the necessary datasets, locate and verify the technology models, and prepare the technology models and data for use in further analyses.

Every decision context has a single Excel workbook containing seven datasets, one per sheet. In no particular order, these datasets are *indices*, *functions*, *designs*, *parameters*, *results*, *tranches*, and *investments*. When the `Designs` and `Investments` classes are instantiated in the code block immediately below, these datasets are automatically checked for internal consistency and completeness. If any of the datasets contain errors or are missing information, the code block below will fail with error messages detailing what data needs to be corrected and in which datasets.

After instantiating the `Designs` class, the `compile` method further processes the numerical datasets to identify any probability distributions in the data and ready them for use in ensemble simulations.

In [ ]:
designs = ty.Designs(path = '.',
                     name = 'template.xlsx')

investments = ty.Investments(path = '.',
                             name = 'template.xlsx')

designs.compile()

## Evaluate the Scenarios

The `evaluate_scenarios` method in the code block below performs ensemble simulation (Monte Carlo) with the number of simulations defined with the `sample_count` parameter. For every simulation, values are sampled from all probability distributions present in the input datasets and are used to calculate corresponding Metric values as well as the built-in Cost. Results of these simulations are returned as shown in the second code block below.

In [ ]:
scenario_results = designs.evaluate_scenarios(sample_count=100)

In [ ]:
scenario_results.xs(1, level="Sample", drop_level=False)

Ensemble simulation results can be further filtered down to show only aggregate results for a single Scenario. This is done in the code block below, in which the complete set of results is filtered down to only the Current State scenario and then average (mean) values are calculated for every Index.

In [ ]:
scenario_results.xs(
    'Current State',
    level='Scenario',
    drop_level=False
).groupby(
    ['Technology', 'Index']
).mean(
    numeric_only=True
)

Here we demonstrate use of the Seaborn visualization package to compare the metric outcomes of each Scenario. Each of the four plots below shows a different metric, quantified as the difference between the Scenario's metric values (one value per simulation) and the average Current State metric value. The plots are put in order of increasing complexity of information presented.

In [ ]:
g = sb.boxplot(
    x="Scenario",
    y="Value",
    hue="Technology",
    data=scenario_results.xs(
        ("Metric", "Delta Labor"),
        level=["Variable", "Index"]
    ).reset_index()[["Technology", "Scenario", "Value"]],

    order=[
        "All Components Slow Progress"      ,
        "All Components Moderate Progress"  ,
        "All Components Fast Progress"      ,
        "Efficiency Slow Progress"    ,
        "Efficiency Moderate Progress",
        "Efficiency Fast Progress"    ,
        "Peripherals Slow Progress"         ,
        "Peripherals Moderate Progress"     ,
        "Peripherals Fast Progress"         ,
    ]
)
g.set(ylabel="Change in Labor per Unit F\nRelative to Current State")
g.set_xticklabels(g.get_xticklabels(), rotation=30, ha='right', rotation_mode='anchor')
g.axhline(y=0, linestyle=":", label='Current State')
g.legend()

In [ ]:
g = sb.boxenplot(
    x="Scenario",
    y="Value",
    hue="Technology",
    scale="area",
    data=scenario_results.xs(
        ("Metric", "Delta System Cost"),
        level=["Variable", "Index"]
    ).reset_index()[["Technology", "Scenario", "Value"]],
    order=[
        "All Components Slow Progress"      ,
        "All Components Moderate Progress"  ,
        "All Components Fast Progress"      ,
        "Efficiency Slow Progress"    ,
        "Efficiency Moderate Progress",
        "Efficiency Fast Progress"    ,
        "Peripherals Slow Progress"         ,
        "Peripherals Moderate Progress"     ,
        "Peripherals Fast Progress"         ,
    ]
)
g.set(ylabel="Change in Annualized System Cost per Unit F\nRelative to Current State")
g.set_xticklabels(g.get_xticklabels(), rotation=30, ha='right', rotation_mode='anchor')
g.axhline(y=0, linestyle=":", label='Current State')
g.legend()

In [ ]:
g = sb.violinplot(
    x="Scenario",
    y="Value",
    hue="Technology",
    dodge=True,
    cut=0,
    data=scenario_results.xs(
        ("Metric", "Delta Overall Efficiency"),
        level=["Variable", "Index"]
    ).reset_index()[["Technology", "Scenario", "Value"]],
    order=[
        "All Components Slow Progress"      ,
        "All Components Moderate Progress"  ,
        "All Components Fast Progress"      ,
        "Efficiency Slow Progress"    ,
        "Efficiency Moderate Progress",
        "Efficiency Fast Progress"    ,
        "Peripherals Slow Progress"         ,
        "Peripherals Moderate Progress"     ,
        "Peripherals Fast Progress"         ,
    ]
)
g.set(ylabel="Change in Overall Efficiency\nRelative to Current State")
g.set_xticklabels(g.get_xticklabels(), rotation=30, ha='right', rotation_mode='anchor')
g.axhline(y=0, linestyle=":", label='Current State')
g.legend()

In [ ]:
g = sb.swarmplot(
    x="Scenario",
    y="Value",
    hue="Technology",
    data=scenario_results.xs(
        ("Metric", "Delta Environment"),
        level=["Variable", "Index"]
    ).reset_index()[["Technology", "Scenario", "Value"]],
    dodge=True,
    size=1.5,
    order=[
        "All Components Slow Progress"      ,
        "All Components Moderate Progress"  ,
        "All Components Fast Progress"      ,
        "Efficiency Slow Progress"    ,
        "Efficiency Moderate Progress",
        "Efficiency Fast Progress"    ,
        "Peripherals Slow Progress"         ,
        "Peripherals Moderate Progress"     ,
        "Peripherals Fast Progress"         ,
    ]
)
g.set(ylabel="Change in Environmental Impact per Unit F\nRelative to Current State")
g.set_xticklabels(g.get_xticklabels(), rotation=30, ha='right', rotation_mode='anchor')
g.axhline(y=0, linestyle=":", label='Current State')
g.legend()

## Evaluate the Investments

In [ ]:
investment_results = investments.evaluate_investments(designs, sample_count=100)

### Costs of investments

In [ ]:
investment_results.amounts

In [ ]:
investment_results.summary.xs(1, level="Sample", drop_level=False)

In [ ]:
g = sb.boxplot(
    x="Investment",
    y="Value",
    hue="Technology",
    data=investment_results.metrics.xs(
        "Delta Overall Efficiency",
        level="Index"
    ).groupby(
        ["Technology","Investment", "Sample"]
    ).sum(
        numeric_only=True
    ).reset_index()[["Technology","Investment", "Value"]],
    order=[
        "Low Budget, All Categories"      ,
        "Low Budget, Components Only"  ,
        "Medium Budget, All Categories"      ,
        "Medium Budget, Components Only"    ,
        "High Budget, All Categories",
        "High Budget, Components Only"
    ]
)
g.set(ylabel="Overall Efficiency")
g.set_xticklabels(g.get_xticklabels(), rotation=30, ha='right', rotation_mode='anchor')
g.axhline(y=0, linestyle=":", label='Current State')
g.legend()

In [ ]:
g = sb.boxenplot(
    x="Investment",
    y="Value",
    hue="Technology",
    data=investment_results.metrics.xs(
        "Delta Environment",
        level="Index"
    ).groupby(
        ["Technology","Investment", "Sample"]
    ).sum(
        numeric_only=True
    ).reset_index()[["Technology","Investment", "Value"]],
    order=[
        "Low Budget, All Categories"      ,
        "Low Budget, Components Only"  ,
        "Medium Budget, All Categories"      ,
        "Medium Budget, Components Only"    ,
        "High Budget, All Categories",
        "High Budget, Components Only"
    ]
)
g.set(ylabel="Environmental Impact per Unit F")
g.set_xticklabels(g.get_xticklabels(), rotation=30, ha='right', rotation_mode='anchor')
g.axhline(y=0, linestyle=":", label='Current State')
g.legend()

In [ ]:
g = sb.violinplot(
    x="Investment",
    y="Value",
    hue="Technology",
    cut=0,
    data=investment_results.metrics.xs(
        "Delta Labor",
        level="Index"
    ).groupby(
        ["Technology","Investment", "Sample"]
    ).sum(
        numeric_only=True
    ).reset_index()[["Technology","Investment", "Value"]],
    order=[
        "Low Budget, All Categories"      ,
        "Low Budget, Components Only"  ,
        "Medium Budget, All Categories"      ,
        "Medium Budget, Components Only"    ,
        "High Budget, All Categories",
        "High Budget, Components Only"
    ]
)
g.set(ylabel="Labor per Unit F")
g.set_xticklabels(g.get_xticklabels(), rotation=30, ha='right', rotation_mode='anchor')
g.axhline(y=0, linestyle=":", label='Current State')
g.legend()

In [ ]:
g = sb.swarmplot(
    x="Investment",
    y="Value",
    hue="Technology",
    size=2,
    dodge=True,
    data=investment_results.metrics.xs(
        "Delta System Cost",
        level="Index"
    ).groupby(
        ["Technology","Investment", "Sample"]
    ).sum(
        numeric_only=True
    ).reset_index()[["Technology","Investment", "Value"]],
    order=[
        "Low Budget, All Categories"      ,
        "Low Budget, Components Only"  ,
        "Medium Budget, All Categories"      ,
        "Medium Budget, Components Only"    ,
        "High Budget, All Categories",
        "High Budget, Components Only"
    ]
)
g.set(ylabel="Annualized System Cost per Unit F")
g.set_xticklabels(g.get_xticklabels(), rotation=30, ha='right', rotation_mode='anchor')
g.axhline(y=0, linestyle=":", label='Current State')
g.legend()

## Multi-objective decision analysis.

### Compute costs and metrics for tranches.

Tranches are atomic units for building investment portfolios. Evaluate all of the tranches, so we can assemble them into investments (portfolios).

In [ ]:
tranche_results = investments.evaluate_tranches(designs, sample_count=100)

Display the cost of each tranche.

In [ ]:
tranche_results.amounts

Display the metrics for each tranche.

In [ ]:
tranche_results.summary

### Fit a response surface to the results.

The response surface interpolates between the discrete set of cases provided in the expert elicitation. This allows us to study funding levels intermediate between those scenarios.

In [ ]:
evaluator = ty.Evaluator(tranche_results)

Here are the categories of investment and the maximum amount that could be invested in each:

In [ ]:
evaluator.max_amount

#### Example interpolation.

Let's evaluate the case where each category is invested in at half of its maximum amount.

In [ ]:
example_investments = evaluator.max_amount / 2
example_investments

In [ ]:
evaluator.evaluate(example_investments)

Let's evaluate the mean instead of outputing the whole distribution.

In [ ]:
evaluator.evaluate_statistic(example_investments, np.mean)

Here is the standard deviation:

In [ ]:
evaluator.evaluate_statistic(example_investments, np.std)

A risk-averse decision maker might be interested in the 10% percentile:

In [ ]:
evaluator.evaluate_statistic(example_investments, lambda x: np.quantile(x, 0.1))